RECUPERATION DE LA LISTE DE TOUS LES ETABLISSEMENTS (AVEC CHANGEMENTS DE PAGE)

In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import plotly.express as px
import pandas as pd
import json
import requests

SI LA LISTE BASIQUE DES ETABLISSEMENTS N'A PAS DEJA ETE ENREGISTREE EN JSON ON VA LA RECUPERER

In [2]:
filename = 'toutes_les_pages.json'

if filename in os.listdir("src/"):
    le_fichier_existe = True
    print("le fichier existe")
    df = pd.DataFrame(pd.read_json('src/'+filename), dtype='str')
else:
    print("Le fichier existe pas")
    le_fichier_existe = False

le fichier existe


ON RECUPERE LA LISTE BASIQUE DES ETABLISSEMENTS (CHANGER EN PYTHON POUR L'UTILISER)

class SFAPSpider(scrapy.Spider):
    # Name of your spider
    name = "sfap"

    # Url to start your spider from
    start_urls = [
        'https://sfap.org/annuaire?contact_sub_type=Structure&type_6%5B%5D=USP&type_6%5B%5D=LISP'
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        # retourne une liste de tous les éléments de classe "quote" 
        entrees = response.css('tbody tr')
        for entree in entrees:
            nom = entree.css('td.views-field-display-name a::text').get()
            type = entree.css('td.views-field-type-6::text').get()
            adresse = entree.css('td.views-field-street-address::text').get()
            code_postal = entree.css('td.views-field-postal-code::text').get()
            commune = entree.css('td.views-field-city::text').get()
            telephone = entree.css('td.views-field-phone::text').get()
            yield {
                'nom': nom,
                'type':type.strip('\n').strip(),
                'adresse':adresse.strip('\n').strip(),
                'code_postal':code_postal.strip('\n').strip(),
                'commune':commune.strip('\n').strip(),
                'telephone':telephone.strip('\n').strip()
            }
        try:
            next_page = response.css('li.pager-next a').attrib['href']
        except:
            logging.info("No next page")
        else:
            yield response.follow(next_page, callback=self.parse)

if le_fichier_existe==False:
    filename = 'toutes_les_pages.json'

    if filename in os.listdir("src/"):
        os.remove('src/'+filename)

    process = CrawlerProcess(settings={
        'USER_AGENT': 'Chrome/97.0',
        'LOG_LEVEL': logging.INFO,
        'FEEDS':{
            'src/' + filename : {"format": "json", 'encoding': 'utf8'},
            },
        "AUTOTHROTTLE_ENABLED": True
    }
    )

    #Lancer le crawler utilisant la spider
    process.crawl(SFAPSpider)
    process.start()

ON VA SUR CHAQUE PAGE POUR RECUPERER DES INFOS SUPPLEMENTAIRES

In [4]:
filename = 'toutes_les_pages_completes.json'

if filename in os.listdir("src/"):
    le_fichier_existe = True
    print("le fichier existe")
    df = pd.DataFrame(pd.read_json('src/'+filename), dtype='str')
else:
    print("Le fichier existe pas")
    le_fichier_existe = False

Le fichier existe pas


SI LE FICHIER N'EXISTE PAS ON SCRAP LES INFOS POUR TOUS LES ETABLISSEMENTS (~ 6 MINUTES)

In [5]:
class SFAPSpider(scrapy.Spider):
    # Name of your spider
    name = "sfap"

    # Url to start your spider from
    start_urls = [
        'https://sfap.org/annuaire?contact_sub_type=Structure&type_6%5B%5D=USP&type_6%5B%5D=LISP'
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        # retourne une liste de tous les établissements
        entrees = response.css('tbody tr')
        for entree in entrees:
            nom = entree.css('td.views-field-display-name a::text').get()
            url = entree.css('td.views-field-display-name a').attrib['href']
            type = entree.css('td.views-field-type-6::text').get()
            adresse = entree.css('td.views-field-street-address::text').get()
            code_postal = entree.css('td.views-field-postal-code::text').get()
            commune = entree.css('td.views-field-city::text').get()
            telephone = entree.css('td.views-field-phone::text').get()
            infos= {
                'nom': nom,
                'type':type.strip('\n').strip(),
                'adresse':adresse.strip('\n').strip(),
                'code_postal':code_postal.strip('\n').strip(),
                'commune':commune.strip('\n').strip(),
                'telephone':telephone.strip('\n').strip()
            }

            yield scrapy.Request("https://sfap.org"+url, callback=self.details, cb_kwargs = {'infos':infos})
                
        try:
            next_page = response.css('li.pager-next a').attrib['href']
        except:
            logging.info("No next page")
        else:
            yield response.follow(next_page, callback=self.parse)

    # ON VA SUR CHAQUE PAGE ET ON CHOPE TOUS LES CHAMPS
    def details(self, response, infos):
        lignes = response.css('div.form-item')
        for ligne in lignes:
            label  = ligne.xpath('./label//text()').get().strip()
            if label in infos:
                label = f"{label}_2"
            valeur = ligne.xpath('./div//text()').get()
            if valeur is None:
                valeur = ligne.css('input::attr(value)').get()

            update= {
                label: valeur
            }
            infos.update(update)
        yield infos


if le_fichier_existe ==False:
    filename = 'toutes_les_pages_completes.json'

    if filename in os.listdir("src/"):
        os.remove('src/'+filename)


    process = CrawlerProcess(settings={
        'USER_AGENT': 'Chrome/97.0',
        'LOG_LEVEL': logging.WARNING,
        'FEEDS':{
            'src/' + filename : {"format": "json", 'encoding': 'utf8'},
            },
        #"AUTOTHROTTLE_ENABLED": True
        }
    )

    #Lancer le crawler utilisant la spider
    process.crawl(SFAPSpider)
    process.start()

2024-11-17 12:55:59 [py.warnings] WARNING: c:\Users\Admin\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2024-11-17 12:58:39 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET https://sfap.org/annuaire/32691> (failed 3 times): 500 Internal Server Error
2024-11-17 12:58:39 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET https://sfap.org/annuaire/32687> (failed 3 times): 500 Internal Server Error
2024-11-17 12:58:39 [scrapy.downl

ON CHARGE LE JSON DANS UN DATAFRAME

In [6]:
# Open and read the JSON file
with open('src/toutes_les_pages_completes.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

ON FAIT UNE COPIE AU CAS OU

In [7]:
df_backup = df

AJOUT DES LATITUDES ET LONGITUDES DE CHAQUE ETABLISSEMENT

In [8]:
filename = 'backup_infos_avec_coordonnees.csv'

if filename in os.listdir("src/"):
    le_fichier_existe = True
    print("le fichier existe")
    df = pd.DataFrame(pd.read_csv('src/'+filename), dtype='str')
else:
    print("Le fichier existe pas")
    le_fichier_existe = False

Le fichier existe pas


SI LE FICHIER CSV N'EXISTE PAS ENCORE ON RECUPERE TOUTES LES COORDONNEES

In [9]:

if le_fichier_existe == False:
    # RECUPERATION DES DONNEES POUR TOUTE LA LISTE DES VILLES
    url = "https://nominatim.openstreetmap.org/search?"
    headers = {'user_agent':'charles.mittempergher@gmail.com'}

    lats = []
    longs = []

    for i in range(0,len(df)):
        street = df.loc[i,'adresse']
        postal_code = df.loc[i,'code_postal']
        p = {'street':{street},'postalcode' : {postal_code},'country':'France', 'format':'json', 'limit':1, 'email':'charles@gmail.com'}
        r = requests.get(url, headers=headers, params=p)

        lat = None
        lon = None
        if len(r.json())>0:
            lats.append(r.json()[0]['lat'])
            longs.append(r.json()[0]['lon'])
        else:
            lats.append(None)
            longs.append(None)

        print(f'{i}/{len(df)} ({round(i/len(df)*100,2)}%)')
        print('*************************')

    df.insert(loc=len(df.columns), column='lat', value=lats)
    df.insert(loc=len(df.columns), column='lon', value=longs)
    df['lat'] = df['lat'].astype('float64')
    df['lon'] = df['lon'].astype('float64')

    df.to_csv("src/backup_infos_avec_coordonnees.csv", index=False)

0/121 (0.0%)
*************************
1/121 (0.83%)
*************************
2/121 (1.65%)
*************************
3/121 (2.48%)
*************************
4/121 (3.31%)
*************************
5/121 (4.13%)
*************************
6/121 (4.96%)
*************************
7/121 (5.79%)
*************************
8/121 (6.61%)
*************************
9/121 (7.44%)
*************************
10/121 (8.26%)
*************************
11/121 (9.09%)
*************************
12/121 (9.92%)
*************************
13/121 (10.74%)
*************************
14/121 (11.57%)
*************************
15/121 (12.4%)
*************************
16/121 (13.22%)
*************************
17/121 (14.05%)
*************************
18/121 (14.88%)
*************************
19/121 (15.7%)
*************************
20/121 (16.53%)
*************************
21/121 (17.36%)
*************************
22/121 (18.18%)
*************************
23/121 (19.01%)
*************************
24/121 (19.83%)
**

ON A RECUPERE TOUS LES CHAMPS, 
TODO:
- RELANCER LE SCRAPPING CAR ERREUR 500 LA DERNIERE FOIS, ON A QUE 121 LIGNES SUR 372
- IL FAUT NETTOYER LES COLONNES NB DES LITS 1 & 2 (ENLEVER LES NAN ET TRANSFORMER EN INT)
- AFFICHER SUR LA CARTE AVEC SIZE = NB LITS TOTAUX
- + d'autres choses....



In [52]:

fig = px.scatter_mapbox(df,lat="lat", lon="lon",  hover_name="nom", height=700,
mapbox_style="open-street-map",title='Carte des lits SP',zoom=4.5,color_continuous_scale=["red","orange","yellowgreen","green"])
fig.update_layout(title_x=0.5)
fig.show()

INFOS DE PAGES PRECISES (ESSAIS)